<a href="https://colab.research.google.com/github/YuriArduino/Estudos_Artificial_Intelligence/blob/Imers%C3%A3o-Agentes-de-IA---Alura/Imers%C3%A3o_Dev_Agentes_IA_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Aula 01: Classificação de intenções com IA

#1 Prepararando o ambiente

##1.2 Dependências

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.4 MB/s eta 0:00:00


##1.3 Imports

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from typing import Literal, List, Dict
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
import sys
import textwrap
import time

##1.4 Conexão com o modelo

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0, # Maior mais criativo, Menor mais objetivo
    )

###Para saber mais: Diferença **google-generativeai** X **langchain_google_genai**


*   **`google-generativeai` (a biblioteca nativa do Google)**: É como o **motor de um carro**. É a peça de engenharia principal, super poderosa, que faz o trabalho fundamental (gerar texto, contar tokens, etc.). Ela te dá acesso direto e total a todas as funcionalidades específicas daquele motor (o Gemini).

*   **LangChain (a biblioteca de orquestração)**: É como o **carro completo construído ao redor do motor**. Ele usa o motor (Gemini), mas adiciona o chassi, o volante, os pedais, o painel e os assentos. Ele torna o motor mais fácil de usar para um propósito maior (dirigir de um ponto A para um ponto B) e o conecta com outras partes (rodas, sistema de som, GPS).

Vamos detalhar as diferenças práticas:

---

### Tabela Comparativa

| Característica | `google-generativeai` (Nativa) | `langchain_google_genai` (Via LangChain) |
| :--- | :--- | :--- |
| **Propósito Principal** | Acesso direto e completo à API do Gemini. | Construir aplicações complexas com LLMs, orquestrando várias etapas. |
| **Nível de Abstração** | **Baixo.** Você interage diretamente com os conceitos da API do Google. | **Alto.** LangChain cria uma "camada de compatibilidade" sobre vários modelos. |
| **Funcionalidades** | Geração de conteúdo, contagem de tokens, embeddings, ajuste de segurança, etc. | **Tudo da nativa, e mais:** Chains, Agentes, Memória, RAG, etc. |
| **Flexibilidade de Modelo** | Feito exclusivamente para os modelos da família Gemini. | **Multi-modelo.** O código que você escreve pode ser facilmente adaptado para usar o GPT-4, Claude, etc. |
| **Facilidade (Tarefas Simples)** | Geralmente mais simples para uma única chamada de API. | Envolve um pouco mais de "boilerplate" (código de configuração) inicial. |
| **Facilidade (Tarefas Complexas)**| Você precisa construir toda a lógica (ex: memória de chat) do zero. | **Muito mais fácil.** Já fornece componentes prontos para tarefas complexas. |

---

### O que isso significa na prática?

#### 1. Abstração e Portabilidade

Com LangChain, o objeto `llm` que você criou é padronizado. Se amanhã você quisesse testar o modelo da OpenAI, você mudaria poucas linhas:

```python
# Com Google
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

# Com OpenAI (exemplo)
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

# O resto do seu código LangChain (chains, agentes) continuaria funcionando igual!
llm.invoke("Me conte uma piada.")
```

Se você usasse a biblioteca nativa, teria que reescrever todo o código de interação com o modelo.

#### 2. O Poder do Ecossistema LangChain

A verdadeira magia de usar LangChain, e o motivo pelo qual é ensinado na imersão, não é apenas para fazer uma pergunta ao modelo. É para construir sistemas mais complexos:

*   **Chains (Correntes):** Você pode criar sequências de tarefas. Por exemplo: "Passo 1: Pegue a pergunta do usuário. Passo 2: Traduza para o inglês. Passo 3: Envie ao Gemini. Passo 4: Pegue a resposta e traduza de volta para o português."
*   **Agentes (Agents):** Você pode dar ferramentas ao LLM. Por exemplo, dar a ele uma ferramenta de busca no Google. Se você perguntar "Qual a previsão do tempo para amanhã?", o agente pode decidir usar a ferramenta de busca, pegar o resultado e só então usar o Gemini para te dar uma resposta em linguagem natural.
*   **RAG (Retrieval-Augmented Generation):** É a técnica mais popular. Você pode fazer o Gemini "conversar com seus documentos". Você fornece uma base de dados (PDFs, sites, etc.), e o LangChain cuida de buscar a informação relevante nesses documentos para que o Gemini possa responder perguntas sobre eles.

### Conclusão

Para a imersão, ficar com **LangChain é a decisão certa**. Você não está apenas aprendendo a usar o Gemini, mas sim a **como construir aplicações robustas em torno de um LLM**, que é a habilidade mais valiosa no mercado hoje.

A contagem de tokens ser `llm.get_num_tokens()` em vez de `model.count_tokens()` é um pequeno sintoma dessa camada de abstração que o LangChain adiciona para tornar tudo mais padronizado e poderoso.

#2 Content





O resultado de `llm.invoke()` não é um texto puro, mas sim um **objeto**. No LangChain, esse objeto é chamado de `AIMessage`.

Este objeto `AIMessage` funciona como um "envelope" que contém várias informações sobre a resposta do modelo:

1.  **.content**: O conteúdo principal, ou seja, o texto da resposta que você queria. É o que está *dentro* do envelope.
2.  **.response_metadata**: Informações extras sobre a execução, como o motivo pelo qual o modelo parou de gerar texto (`finish_reason`) e as classificações de segurança (`safety_ratings`).
3.  **.usage_metadata**: Dados sobre o consumo de tokens na chamada (quantos tokens na entrada, quantos na saída e o total).
4.  **.id**: Um identificador único para aquela execução específica, útil para depuração.


### Comparando com o `StrOutputParser`

Agora você entende perfeitamente a diferença:

*   **Usar `.content`**: É a forma manual e direta de extrair o texto de uma **única** chamada. É perfeito para testes rápidos e quando você não está construindo uma sequência complexa de passos.

*   **Usar `| StrOutputParser()`**: É a forma automática de fazer a mesma coisa, mas de um jeito que se integra com o sistema de "correntes" (chains) do LangChain. Ele já "desempacota" o conteúdo para você e o entrega pronto para o próximo passo da sua chain.


In [4]:
resp_test = llm.invoke("Estamos Online?")
print(resp_test)

content='Olá! Sim, estou online e pronto para conversar.\n\nComo posso ajudar?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--075bdcf3-d860-4838-b4e3-6717b8ce848b-0' usage_metadata={'input_tokens': 4, 'output_tokens': 16, 'total_tokens': 289, 'input_token_details': {'cache_read': 0}}


In [5]:
#.content organiza o texto
resp_test = llm.invoke("Estamos Online?")
print(resp_test.content)

¡Sí, estamos online! ¿En qué puedo ayudarte?


#3 Tokens

Para um modelo de linguagem como o Gemini, os **tokens são os blocos de construção do texto**.

Um token não é exatamente uma palavra, uma letra ou uma sílaba. É a unidade fundamental que o modelo usa para processar e entender a linguagem.

*   Uma palavra curta como `"Eu"` pode ser **1 token**.
*   Uma palavra comum como `"gato"` pode ser **1 token**.
*   Uma palavra mais longa ou incomum como `"inteligência"` pode ser dividida em **2 ou mais tokens** (ex: `inteli` + `gência`).
*   Pontuação e espaços também contam e podem ser tokens.

**Exemplo Rápido:**
A frase `Olá, mundo!` é quebrada pelo modelo em algo como: `["Olá", ",", " mundo", "!"]` - resultando em **4 tokens**.

Em resumo, para o modelo, toda a nossa conversa (tanto o que enviamos quanto o que ele responde) é uma sequência de tokens.

### A Importância de um Contador de Tokens na nossa Imersão

No desenvolvimento de software, todo recurso é finito. Contar tokens é uma prática de **gerenciamento de recursos computacionais**. Ignorar essa métrica é como escrever um código que consome memória sem controle. A ferramenta `count_querys` que construímos é crucial por três razões técnicas:

**1. Gerenciamento de Custos e Carga de API:**
   Cada token processado representa uma unidade de computação que tem um custo. Em um ambiente de produção, esse custo é financeiro. Na Imersão, ele se reflete no consumo da nossa cota gratuita. O contador funciona como um *profiler*, permitindo-nos:
   *   Medir a "carga" exata que cada chamada impõe à API.
   *   Otimizar a comunicação para ser mais eficiente e evitar exceder os limites de requisições por minuto (*rate limits*).

**2. Respeitar a Janela de Contexto (Payload Capacity):**
   Todo modelo de linguagem opera com uma "janela de contexto" finita – a quantidade máxima de tokens que ele pode considerar em uma única requisição. Essa janela é a memória de trabalho da operação.
   *   Enviar um `payload` (dados de entrada) que exceda essa capacidade resultará em erro.
   *   Nosso contador é uma ferramenta de validação prévia, garantindo que nossas requisições sejam sempre viáveis e estejam dentro das especificações do modelo.

**3. Otimização de Prompts e Latência:**
   A engenharia de prompts é uma disciplina de otimização. Um prompt eficiente transmite a instrução máxima com o mínimo de tokens.
   *   **Sinal vs. Ruído:** Tokens irrelevantes são "ruído" que pode degradar a qualidade da resposta e aumentar a latência (tempo de resposta).
   *   Ao monitorar a contagem, aprendemos a construir prompts mais concisos e potentes, melhorando tanto a performance da aplicação quanto a precisão dos resultados.

---

In [6]:
# 1. Vamos primeiro gerar um objeto de resposta real
resposta_obj = llm.invoke("Estamos online?")
print("--- Objeto Completo ---")
print(resposta_obj)
print("\n" + "="*40 + "\n")


# 2. Agora, vamos isolar cada parte.

# --- Acessando o CONTEÚDO ---
# O conteúdo principal é um atributo direto do objeto.
conteudo_texto = resposta_obj.content
print("--- Conteúdo Isolado (str) ---")
print(conteudo_texto)
print(f"Tipo: {type(conteudo_texto)}\n")


# --- Acessando os METADADOS DE USO ---
# O usage_metadata é um atributo que contém um dicionário.
meta_uso = resposta_obj.usage_metadata
print("--- Metadados de Uso Isolados (dict) ---")
print(meta_uso)
print(f"Tipo: {type(meta_uso)}\n")


# --- Acessando os VALORES DENTRO DOS METADADOS DE USO ---
# Agora que temos o dicionário 'meta_uso', podemos pegar cada valor pela sua chave.
# Usar .get('chave', 0) é mais seguro, pois retorna 0 se a chave não existir.
input_tokens = meta_uso.get('input_tokens', 0)
output_tokens = meta_uso.get('output_tokens', 0)
total_tokens = meta_uso.get('total_tokens', 0)

print("--- Tokens Isolados (int) ---")
print(f"Tokens de Entrada: {input_tokens}")
print(f"Tokens de Saída: {output_tokens}")
print(f"Tokens Totais: {total_tokens}\n")


# --- Acessando OUTROS METADADOS ---
# O mesmo princípio se aplica a outros atributos.
id_da_execucao = resposta_obj.id
motivo_finalizacao = resposta_obj.response_metadata.get('finish_reason', 'N/A')

print("--- Outros Metadados Isolados ---")
print(f"ID da Execução: {id_da_execucao}")
print(f"Motivo da Finalização: {motivo_finalizacao}\n")

--- Objeto Completo ---
content='¡Claro que sí! Estoy aquí y listo para ayudarte.\n\n¿En qué puedo asistirte hoy?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run--a15dfa66-57a3-471b-8075-53344c5133ac-0' usage_metadata={'input_tokens': 4, 'output_tokens': 22, 'total_tokens': 740, 'input_token_details': {'cache_read': 0}}


--- Conteúdo Isolado (str) ---
¡Claro que sí! Estoy aquí y listo para ayudarte.

¿En qué puedo asistirte hoy?
Tipo: <class 'str'>

--- Metadados de Uso Isolados (dict) ---
{'input_tokens': 4, 'output_tokens': 22, 'total_tokens': 740, 'input_token_details': {'cache_read': 0}}
Tipo: <class 'dict'>

--- Tokens Isolados (int) ---
Tokens de Entrada: 4
Tokens de Saída: 22
Tokens Totais: 740

--- Outros Metadados Isolados ---
ID da Execução: run--a15dfa66-57a3-471b-8075-53344c5133ac-0
Motivo da Finalização: STOP



## 3.1 Utilitário de Desenvolvimento: Profiler de Custo para API Gemini

In [7]:
class count_querys:
    def __init__(self):
        self.total_tokens = 0
        print(" Count de querys ativo!")

    def processar_query(self, llm):
        """Processa uma query com count automático e formatação de saída."""
        query = input("Digite seu texto: ")

        if not query.strip():
            print("Nenhum texto inserido.")
            return

        # Envia para o LLM
        resposta = llm.invoke(query)

        # Conta os tokens desta query
        tokens_desta_msg = resposta.usage_metadata.get('total_tokens', 0) if resposta.usage_metadata else 0
        self.total_tokens += tokens_desta_msg

        # Formata a resposta para ter no máximo 90 caracteres por linha
        prefixo = " Resposta: "
        largura_maxima = 90

        # A função fill quebra o texto em linhas do tamanho desejado
        # e adiciona um recuo nas linhas seguintes para manter o alinhamento.
        texto_formatado = textwrap.fill(
            resposta.content,
            width=largura_maxima,
            initial_indent=prefixo,
            subsequent_indent=' ' * len(prefixo) # Recuo para alinhar com o início do texto
        )

        # Mostra a resposta da IA já formatada
        print(f"\n{texto_formatado}")

        # Mostra o count no final
        print(f"\n {tokens_desta_msg} tokens, total {self.total_tokens}")
# Crie o count uma vez
count = count_querys()

def query():
    global count
    if 'count' not in globals():
        count = count_querys()
    count.processar_query(llm)

# Status do count
def status():
    """Mostra o status atual dos tokens"""
    global count
    if 'count' not in globals():
        print("❌ Nenhuma conversa iniciada ainda. Use query() primeiro!")
    else:
        print(f" Total de tokens usados: {count.total_tokens}")

def reset():
    """Reseta o count de tokens"""
    global count
    count = count_querys()
    print(" count resetado!")

def help_chat():
  """Mostra as utilidades do chat."""
  print("""
UTILIDADES:
  query()     - Enviar mensagem
  status()     - Ver total tokens
  reset()      - Resetar count
  help_chat()  - Esta ajuda
""")

 Count de querys ativo!


In [8]:
query()

Digite seu texto: exit

 Resposta: Are you trying to end our conversation? If you have any more questions or need
           help with anything else, feel free to ask. Otherwise, have a great day!

 663 tokens, total 663


In [9]:
status()

 Total de tokens usados: 663


In [10]:
reset()
print(status())

 Count de querys ativo!
 count resetado!
 Total de tokens usados: 0
None


In [11]:
help_chat()


UTILIDADES:
  query()     - Enviar mensagem
  status()     - Ver total tokens
  reset()      - Resetar count
  help_chat()  - Esta ajuda



#4 Estruturando a Saída com Pydantic



Para um programa de computador, processar variações  é propenso a erros. É importante uma estrutura de dados confiável, não um texto para interpretar.

### **Pydantic como um "Contrato" de Dados**

**Pydantic** é uma biblioteca Python para **validação de dados**. Ela nos permite definir "esquemas" ou "modelos" de dados usando classes Python. Em outras palavras, criamos um **contrato** que define a estrutura e os tipos de dados que esperamos receber.

O LangChain se integra perfeitamente com o Pydantic através do **`PydanticOutputParser`**. Este "parser" (analisador) faz duas coisas mágicas:

1.  **Instrui o LLM:** Ele analisa nosso modelo Pydantic e gera instruções claras de formatação que são enviadas ao LLM junto com nosso prompt.
2.  **Valida a Resposta:** Ele pega a resposta em texto do LLM, verifica se ela segue o "contrato", e a converte em um objeto Python real e validado.


###4.1 Configuração Inicial e Imports

In [12]:
# Imports de bibliotecas de tipagem e validação
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

# Imports principais do LangChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage

# Configuração da API Key (maneira segura para o Colab)
import os
from google.colab import userdata

# Garanta que a variável de ambiente está configurada
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')

>Esta célula inicial é responsável por preparar nosso ambiente de trabalho.
>
>Importamos todas as bibliotecas necessárias, incluindo o Pydantic para definir nossa estrutura de dados e o LangChain para orquestrar a comunicação com o modelo Gemini.
>
>Também configuramos nossa chave de API do Google de forma segura usando os segredos do Colab.

##4.2 Definindo o "Cérebro" do Agente (O Prompt Principal)

In [13]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

>Aqui definimos o SystemMessage, a personalidade e as diretrizes do nosso agente de IA.
>
>Este prompt é o "cérebro" da operação, explicando ao modelo seu papel, as regras de negócio para cada tipo de decisão e o que esperamos como resultado.

##4.3 Definindo o "Contrato" de Saída (O Esquema Pydantic)

In [14]:
class TriagemOut(BaseModel):
    """Define a estrutura de saída esperada para a triagem de chamados."""

    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"] = Field(
        description="A decisão principal baseada na mensagem do usuário."
    )
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"] = Field(
        description="A urgência estimada do chamado."
    )
    campos_faltantes: List[str] = Field(
        default_factory=list,
        description="Uma lista de informações que o usuário precisa fornecer se a decisão for 'PEDIR_INFO'."
    )

>Esta é a parte central da validação. Usando Pydantic, criamos a classe TriagemOut que funciona como um "contrato".
>
>Ela força o LLM a gerar uma resposta que contenha exatamente os campos decisao, urgencia e campos_faltantes, com os tipos e valores que definimos. Se a IA gerar algo fora deste padrão, o LangChain acusará um erro.

##4.4 Célula Principal - Construindo e Executando a Lógica de Triagem

In [15]:
# 1. Inicializa o LLM com temperatura 0.0 para respostas mais determinísticas
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # Usando um modelo padrão e eficiente para essa tarefa
    temperature=0.0
)

# 2. Cria a "chain" que conecta o LLM ao nosso esquema de saída Pydantic
# O método .with_structured_output() é o que faz a mágica acontecer!
triagem_chain = llm_triagem.with_structured_output(TriagemOut)

# 3. Define a função que invoca a chain e formata a saída
def triagem(mensagem: str) -> Dict:
    """
    Invoca a chain de triagem com a mensagem do usuário e retorna um dicionário.
    """
    # A chain recebe uma lista de mensagens (Sistema e Usuário)
    saida_pydantic: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    # .model_dump() converte o objeto Pydantic validado em um dicionário Python
    return saida_pydantic.model_dump()

>Esta é a célula principal onde a orquestração acontece. Nós:

1)   Instanciamos o modelo Gemini, configurando a temperature para 0.0 para obter resultados mais consistentes e previsíveis.
2)   Criamos a triagem_chain usando o poderoso método .with_structured_output(TriagemOut). É este comando que "ensina" o LLM a seguir nosso esquema Pydantic.
3)   Definimos a função triagem(), que encapsula a lógica de chamada, enviando o prompt do sistema e a mensagem do usuário para a chain.

##4.5 Teste e Validação

In [16]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?",
          "Tenho uma dúvida sobre um benefício."]

for msg_teste in testes:
    print(f"Pergunta: '{msg_teste}'")
    resultado = triagem(msg_teste)
    print(f" -> Resposta Estruturada: {resultado}\n")

Pergunta: 'Posso reembolsar a internet?'
 -> Resposta Estruturada: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: 'Quero mais 5 dias de trabalho remoto. Como faço?'
 -> Resposta Estruturada: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

Pergunta: 'Posso reembolsar cursos ou treinamentos da Alura?'
 -> Resposta Estruturada: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: 'Quantas capivaras tem no Rio Pinheiros?'
 -> Resposta Estruturada: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['Sua pergunta não está relacionada às políticas internas da empresa Carraro Desenvolvimento.']}

Pergunta: 'Tenho uma dúvida sobre um benefício.'
 -> Resposta Estruturada: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['Qual benefício você se refere?']}



##4.6 **Teste e Validação(Com Controle de Taxa)**

> **Nota Importante sobre Limites de API (Rate Limiting)**
>
> Ao executar o loop `for` sem pausas, enviamos todas as nossas perguntas à API quase simultaneamente. Isso aciona um mecanismo de proteção da API do Google chamado **rate limiting**, resultando no erro `ResourceExhausted: 429`. Essencialmente, a API nos diz: "Por favor, vá com mais calma!".
>
> Para resolver isso, introduzimos o comando **`time.sleep(10)`** dentro do loop. Este comando pausa a execução por 10 segundos entre cada chamada, garantindo que respeitamos os limites da camada gratuita (ex: 2-5 requisições por minuto).
>
> Gerenciar a frequência das chamadas é uma **prática fundamental** no desenvolvimento com qualquer serviço externo, garantindo que nossa aplicação seja robusta e não seja bloqueada.

In [18]:
# Importamos a biblioteca 'time' para adicionar pausas
import time

testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?",
          "Tenho uma dúvida sobre um benefício."]

for msg_teste in testes:
    print(f"Pergunta: '{msg_teste}'")
    resultado = triagem(msg_teste)
    print(f" -> Resposta Estruturada: {resultado}\n")

    # --- AJUSTE IMPORTANTE AQUI ---
    # Adiciona uma pausa de 10 segundos para não exceder o limite da API.
    # Para o free tier, um valor entre 10 e 20 segundos é seguro.
    print("...aguardando 12 segundos para a próxima requisição...\n")
    time.sleep(12)

print("✅ Todos os testes foram concluídos.")

Pergunta: 'Posso reembolsar a internet?'
 -> Resposta Estruturada: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

...aguardando 10 segundos para a próxima requisição...

Pergunta: 'Quero mais 5 dias de trabalho remoto. Como faço?'
 -> Resposta Estruturada: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}

...aguardando 10 segundos para a próxima requisição...

Pergunta: 'Posso reembolsar cursos ou treinamentos da Alura?'
 -> Resposta Estruturada: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

...aguardando 10 segundos para a próxima requisição...

Pergunta: 'Quantas capivaras tem no Rio Pinheiros?'
 -> Resposta Estruturada: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['Por favor, especifique como sua pergunta se relaciona com as políticas internas da empresa Carraro Desenvolvimento.']}

...aguardando 10 segundos para a próxima requisição...

Pergunta: 'Tenho uma dúvida sobre um benef

>Com tudo configurado, esta célula executa um conjunto de testes para validar nosso sistema. Observamos como ele classifica diferentes tipos de perguntas, demonstrando a eficácia do prompt e da estruturação de saída com Pydantic.

# Aula 02: Construindo a base de conhecimento com RAG

##Install

In [21]:
!pip install -q --upgrade langchain_community faiss-cpu langchain-text-splitters pymupdf

## Lista de URLs

In [42]:
urls = [
    "https://github.com/YuriArduino/Estudos_Artificial_Intelligence/blob/Dados/Pol%C3%ADtica%20de%20Reembolsos%20(Viagens%20e%20Despesas).pdf",
    "https://github.com/YuriArduino/Estudos_Artificial_Intelligence/blob/Dados/Pol%C3%ADtica%20de%20Uso%20de%20E-mail%20e%20Seguran%C3%A7a%20da%20Informa%C3%A7%C3%A3o.pdf",
    "https://github.com/YuriArduino/Estudos_Artificial_Intelligence/blob/Dados/Pol%C3%ADticas%20de%20Home%20Office.pdf"
]

Imports

In [46]:
import requests
import os
from pathlib import Path
from urllib.parse import unquote
import tempfile
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

#Função para Carregar Documentos

In [80]:
import requests
import os
from pathlib import Path
from urllib.parse import unquote # Import unquote for decoding URLs
import tempfile # Import tempfile for safer temporary file handling

# Removed the global 'docs' initialization here as it's done outside the function now

def carregar_documentos_de_urls(lista_de_urls: list) -> list:
    """
    Baixa, decodifica e carrega documentos PDF de uma lista de URLs.
    Retorna uma lista de documentos do LangChain.
    """
    print("Iniciando o carregamento dos documentos...")
    documentos_carregados = []

    for url in lista_de_urls:
        nome_arquivo_decodificado = "" # Initialize here
        try:
            # Decodifica o nome do arquivo para uma exibição limpa
            nome_arquivo_decodificado = unquote(url.split('/')[-1])
            print(f"🔄 Processando: '{nome_arquivo_decodificado}'...")

            # Constrói o URL para o download do arquivo raw
            url_raw = url + '?raw=true'

            # Usa um arquivo temporário que é deletado automaticamente
            with tempfile.NamedTemporaryFile(suffix=".pdf", delete=False) as temp_file:
                temp_file_path = temp_file.name # Store the actual temp file path
                # Faz o download do conteúdo
                response = requests.get(url_raw, timeout=20)
                response.raise_for_status()  # Lança erro para status HTTP ruins

                # Escreve o conteúdo no arquivo temporário
                temp_file.write(response.content)
                # temp_file.flush() # Garante que tudo foi escrito no disco (not strictly needed with 'with')


            # Usa o PyMuPDFLoader para carregar do arquivo temporário
            loader = PyMuPDFLoader(temp_file_path)
            paginas = loader.load()

            # Add the decoded filename to the metadata of each page
            for pagina in paginas:
                pagina.metadata['decoded_filename'] = nome_arquivo_decodificado
                documentos_carregados.append(pagina)

            print(f"✅ Sucesso: '{nome_arquivo_decodificado}' carregado.")

        except requests.RequestException as e:
            print(f"❌ Erro de Rede ao carregar '{nome_arquivo_decodificado}': {e}")
        except Exception as e:
            print(f"❌ Erro inesperado ao processar '{nome_arquivo_decodificado}': {e}")
        finally:
            # Ensure the temporary file is removed even if errors occur
            if 'temp_file_path' in locals() and os.path.exists(temp_file_path):
                os.remove(temp_file_path)


    return documentos_carregados

# --- 3. Execução ---
# Ensure 'urls' variable exists, assuming it's defined elsewhere
# If not, you might need to define it here or ensure the user runs the cell above
# Example: urls = ["...", "...", "..."]
docs = carregar_documentos_de_urls(urls) # Use the 'urls' variable defined previously

print("\n" + "="*50)
print("Resumo do Carregamento:")
print(f"  - Total de fontes (URLs) processadas: {len(urls)}")
print(f"  - Total de páginas carregadas (documentos): {len(docs)}")
print("="*50)

Iniciando o carregamento dos documentos...
🔄 Processando: 'Política de Reembolsos (Viagens e Despesas).pdf'...
✅ Sucesso: 'Política de Reembolsos (Viagens e Despesas).pdf' carregado.
🔄 Processando: 'Política de Uso de E-mail e Segurança da Informação.pdf'...
✅ Sucesso: 'Política de Uso de E-mail e Segurança da Informação.pdf' carregado.
🔄 Processando: 'Políticas de Home Office.pdf'...
✅ Sucesso: 'Políticas de Home Office.pdf' carregado.

Resumo do Carregamento:
  - Total de fontes (URLs) processadas: 3
  - Total de páginas carregadas (documentos): 3


### Verificando o conteúdo de uma página

In [45]:

if docs:
    print("\nExemplo de conteúdo da primeira página carregada:")
    print(docs[0].page_content[:400] + "...")


Exemplo de conteúdo da primeira página carregada:
Política de Reembolsos (Viagens e 
Despesas) 
 
1.​ Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a 
despesa.​
 
2.​ Alimentação em viagem: limite de R$ 70/dia por pessoa. Bebidas alcoólicas não 
são reembolsáveis.​
 
3.​ Transporte: táxi/app são permitidos quando não houver alternativa viável. 
Comprovantes obrigatórios.​
 
4.​ Internet para home office: reembols...


In [115]:
#from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)

chunks = splitter.split_documents(docs)

In [116]:
for chunk in chunks:
    print(chunk)
    print("------------------------------------")

total_chunk = len(chunks)
print(f"Total de Chunks: {total_chunk}")

page_content='Política de Reembolsos (Viagens e 
Despesas)' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/tmp/tmpyyd6m4ul.pdf', 'file_path': '/tmp/tmpyyd6m4ul.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0, 'decoded_filename': 'Política de Reembolsos (Viagens e Despesas).pdf'}
------------------------------------
page_content='Despesas) 
 
1.​ Reembolso: requer nota fiscal e deve ser submetido em até 10 dias corridos após a' metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/tmp/tmpyyd6m4ul.pdf', 'file_path': '/tmp/tmpyyd6m4ul.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '

In [133]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
)

In [135]:
# Célula para definir nossa ferramenta de análise

def analisar_qualidade_retriever(vectorstore, llm, query_de_teste: str, k_inicial: int = 5):
    """
    Executa um pipeline completo de análise de retriever:
    1. Faz uma busca vetorial inicial para obter N candidatos.
    2. Usa um "LLM Juiz" para validar a relevância de cada candidato.
    3. Usa um "LLM Analista" para analisar os resultados e sugerir os melhores
       parâmetros `k` e `score_threshold`.
    """
    print("\n" + "="*80)
    print("🚀 Iniciando pipeline de análise de qualidade do retriever...")
    print(f"Pergunta de Teste: '{query_de_teste}'")
    print("="*80)

    # --- ETAPA 1: Busca Vetorial Inicial (Generosa) ---
    print(f"\n1. Buscando os {k_inicial} melhores candidatos no Vector Store...")

    documentos_com_score = vectorstore.similarity_search_with_score(query_de_teste, k=k_inicial)

    if not documentos_com_score:
        print("❌ Nenhum documento encontrado na busca vetorial. Fim da análise.")
        return

    # --- ETAPA 2: Validação com "LLM Juiz" ---
    print("\n2. Usando 'LLM Juiz' para validar a relevância de cada candidato...")

    resultados_analise = []
    for i, (doc, score) in enumerate(documentos_com_score):
        conteudo_chunk = doc.page_content.replace('\n', ' ')

        prompt_juiz = f"""
        Avalie a relevância do CONTEXTO para responder à PERGUNTA.
        Responda APENAS com a palavra 'SIM' ou 'NÃO'.

        PERGUNTA: "{query_de_teste}"
        CONTEXTO: "{conteudo_chunk}"

        AVALIAÇÃO (SIM/NÃO):
        """

        print(f"--- Avaliando Chunk {i+1} (Score: {score:.4f}) ---")
        resposta_juiz = llm.invoke(prompt_juiz)
        decisao = "SIM" in resposta_juiz.content.strip().upper()

        print(f"Resultado do Juiz: {'✅ RELEVANTE' if decisao else '❌ IRRELEVANTE'}")

        resultados_analise.append({
            "id": i + 1,
            "score": score,
            "relevante": decisao
        })

    # --- ETAPA 3: Análise Final com "LLM Analista" ---
    print("\n3. Usando 'LLM Analista' para sugerir os melhores parâmetros...")

    relatorio_str = ""
    for res in resultados_analise:
        relatorio_str += f"- Candidato {res['id']}: Score de Distância = {res['score']:.4f}, Julgamento de Relevância = {'SIM' if res['relevante'] else 'NÃO'}\n"

    prompt_analista = f"""
    Você é um especialista em otimização de sistemas de RAG. Sua tarefa é analisar os resultados de uma busca vetorial e da validação de um LLM Juiz para sugerir os melhores parâmetros `k` e `score_threshold`.

    REGRAS IMPORTANTES:
    - O score é a Distância L2, onde **MENOR é MELHOR**.
    - `k` é o número máximo de documentos a serem recuperados.
    - `score_threshold` é o limite máximo de distância. Documentos com score MAIOR que o threshold serão descartados.

    DADOS DA ANÁLISE:
    {relatorio_str}

    SUA SUGESTÃO:
    Com base na análise, forneça uma sugestão clara para `k` e `score_threshold`. Justifique sua resposta brevemente.
    """

    sugestao_final = llm.invoke(prompt_analista)

    print("\n" + "="*80)
    print("🤖 SUGESTÃO FINAL DO ESPECIALISTA EM RAG 🤖\n")
    print(sugestao_final.content)
    print("="*80)

In [136]:
# --- Experimento 1: Chunks Pequenos (100/50) ---
print("="*80)
print("🧪 Iniciando Experimento com Estratégia 'Pequeno e Preciso' (100/50)")
print("="*80)

# 1. Dividir os documentos
splitter_pequeno = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
chunks_pequenos = splitter_pequeno.split_documents(docs)
print(f"Total de chunks gerados: {len(chunks_pequenos)}")

# 2. Gerar Embeddings e Criar o Vector Store (CUSTO DE API ACONTECE AQUI)
print("🔄 Criando Vector Store e gerando embeddings...")
vectorstore_pequeno = FAISS.from_documents(chunks_pequenos, embeddings)
print("✅ Vector Store 'pequeno' criado com sucesso.")

# 3. Analisar e Validar a Qualidade
query_de_teste = "Posso reembolsar cursos ou treinamentos da Alura?"
analisar_qualidade_retriever(vectorstore_pequeno, llm, query_de_teste)

# --- Experimento 2: Chunks Médios (250/50) ---
print("\n" + "="*80)
print("🧪 Iniciando Experimento com Estratégia 'Médio e Equilibrado' (250/50)")
print("="*80)

# 1. Dividir os documentos
splitter_medio = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
chunks_medios = splitter_medio.split_documents(docs)
print(f"Total de chunks gerados: {len(chunks_medios)}")

# 2. Gerar Embeddings e Criar o Vector Store
print("🔄 Criando Vector Store e gerando embeddings...")
vectorstore_medio = FAISS.from_documents(chunks_medios, embeddings)
print("✅ Vector Store 'medio' criado com sucesso.")

# 3. Analisar e Validar a Qualidade
query_de_teste = "Posso reembolsar cursos ou treinamentos da Alura?"
analisar_qualidade_retriever(vectorstore_medio, llm, query_de_teste)

🧪 Iniciando Experimento com Estratégia 'Pequeno e Preciso' (100/50)
Total de chunks gerados: 34
🔄 Criando Vector Store e gerando embeddings...
✅ Vector Store 'pequeno' criado com sucesso.

🚀 Iniciando pipeline de análise de qualidade do retriever...
Pergunta de Teste: 'Posso reembolsar cursos ou treinamentos da Alura?'

1. Buscando os 5 melhores candidatos no Vector Store...

2. Usando 'LLM Juiz' para validar a relevância de cada candidato...
--- Avaliando Chunk 1 (Score: 0.4932) ---
Resultado do Juiz: ✅ RELEVANTE
--- Avaliando Chunk 2 (Score: 0.6641) ---
Resultado do Juiz: ❌ IRRELEVANTE
--- Avaliando Chunk 3 (Score: 0.6947) ---
Resultado do Juiz: ❌ IRRELEVANTE
--- Avaliando Chunk 4 (Score: 0.6970) ---
Resultado do Juiz: ❌ IRRELEVANTE
--- Avaliando Chunk 5 (Score: 0.7105) ---
Resultado do Juiz: ❌ IRRELEVANTE

3. Usando 'LLM Analista' para sugerir os melhores parâmetros...

🤖 SUGESTÃO FINAL DO ESPECIALISTA EM RAG 🤖

Com base na análise dos dados, sugiro os seguintes parâmetros:

**SUA S

In [126]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                     search_kwargs={"score_threshold":0.62, "k": 5})

In [128]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_rag = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um Assistente de Políticas Internas (RH/IT) da empresa Carraro Desenvolvimento. "
     "Responda SOMENTE com base no contexto fornecido. "
     "Se não houver base suficiente, responda apenas 'Não sei'."),

    ("human", "Pergunta: {input}\n\nContexto:\n{context}")
])

document_chain = create_stuff_documents_chain(llm_triagem, prompt_rag)

In [129]:
# Formatadores
import re, pathlib

def _clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s or "").strip()

def extrair_trecho(texto: str, query: str, janela: int = 240) -> str:
    txt = _clean_text(texto)
    termos = [t.lower() for t in re.findall(r"\w+", query or "") if len(t) >= 4]
    pos = -1
    for t in termos:
        pos = txt.lower().find(t)
        if pos != -1: break
    if pos == -1: pos = 0
    ini, fim = max(0, pos - janela//2), min(len(txt), pos + janela//2)
    return txt[ini:fim]

def formatar_citacoes(docs_rel: List, query: str) -> List[Dict]:
    cites, seen = [], set()
    for d in docs_rel:
        src = pathlib.Path(d.metadata.get("source","")).name
        page = int(d.metadata.get("page", 0)) + 1
        key = (src, page)
        if key in seen:
            continue
        seen.add(key)
        cites.append({"documento": src, "pagina": page, "trecho": extrair_trecho(d.page_content, query)})
    return cites[:3]

In [85]:
def perguntar_politica_RAG(pergunta: str) -> Dict:
    docs_relacionados = retriever.invoke(pergunta)

    if not docs_relacionados:
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    answer = document_chain.invoke({"input": pergunta,
                                    "context": docs_relacionados})

    txt = (answer or "").strip()

    if txt.rstrip(".!?") == "Não sei":
        return {"answer": "Não sei.",
                "citacoes": [],
                "contexto_encontrado": False}

    return {"answer": txt,
            "citacoes": formatar_citacoes(docs_relacionados, pergunta),
            "contexto_encontrado": True}

In [88]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [131]:
for msg_teste in testes:
    resposta = perguntar_politica_RAG(msg_teste)
    print(f"PERGUNTA: {msg_teste}")
    print(f"RESPOSTA: {resposta['answer']}")
    if resposta['contexto_encontrado']:
        print("CITAÇÕES:")
        for c in resposta['citacoes']:
            print(f" - Documento: {c['documento']}, Página: {c['pagina']}")
            print(f"   Trecho: {c['trecho']}")
        print("------------------------------------")

PERGUNTA: Posso reembolsar a internet?
RESPOSTA: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100.
CITAÇÕES:
 - Documento: tmpyyd6m4ul.pdf, Página: 1
   Trecho: são reembolsáveis.​
------------------------------------
PERGUNTA: Quero mais 5 dias de trabalho remoto. Como faço?
RESPOSTA: Deve ser formalizada via chamado.
CITAÇÕES:
 - Documento: tmps31mrujc.pdf, Página: 1
   Trecho: 6.​ Solicitação de exceção (ex.: 4-5 dias remotos): deve ser formalizada via chamado
------------------------------------


PERGUNTA: Posso reembolsar cursos ou treinamentos da Alura?
RESPOSTA: Sim, são reembolsáveis.
CITAÇÕES:
 - Documento: tmpyyd6m4ul.pdf, Página: 1
   Trecho: são reembolsáveis.​
------------------------------------
PERGUNTA: Quantas capivaras tem no Rio Pinheiros?
RESPOSTA: Não sei.


````

PERGUNTA: Posso reembolsar a internet?
RESPOSTA: Sim, a internet para home office é reembolsável via subsídio mensal de até R$ 100, requerendo nota fiscal.
CITAÇÕES:
 - Documento: tmpajz7rgts.pdf, Página: 1
   Trecho: são reembolsáveis.​
------------------------------------
PERGUNTA: Quero mais 5 dias de trabalho remoto. Como faço?
RESPOSTA: Você deve formalizar a solicitação via chamado. A exceção precisa ser aprovada pelo seu gestor e pelo RH.
CITAÇÕES:
 - Documento: tmpjz9oxrgs.pdf, Página: 1
   Trecho: (ex.: 4-5 dias remotos): deve ser formalizada via chamado
 - Documento: tmpajz7rgts.pdf, Página: 1
   Trecho: conforme política de Home Office.​
------------------------------------
PERGUNTA: Posso reembolsar cursos ou treinamentos da Alura?
RESPOSTA: Não sei.
PERGUNTA: Quantas capivaras tem no Rio Pinheiros?
RESPOSTA: Não sei.

````

In [132]:
import plotly.express as px
from sklearn.decomposition import PCA
import numpy as np
import textwrap
import os

# Get the text content and source filename from the chunks
chunk_texts = []
# Modify to get the decoded filename from metadata
chunk_sources = []
for chunk in chunks:
    chunk_texts.append(chunk.page_content)
    # Extract the decoded filename from the chunk metadata
    decoded_name = chunk.metadata.get("decoded_filename", "Unknown Source")
    chunk_sources.append(decoded_name)


# Create embeddings for the chunk texts
# Note: This might take some time depending on the number of chunks
chunk_embeddings = embeddings.embed_documents(chunk_texts)

# Convert embeddings to a numpy array
X = np.array(chunk_embeddings)

# Apply PCA for dimensionality reduction to 3 components
pca = PCA(n_components=3)
components = pca.fit_transform(X)

# Calculate the total explained variance
total_var = pca.explained_variance_ratio_.sum() * 100

# Create a DataFrame for Plotly
import pandas as pd
df_plot = pd.DataFrame(components, columns=['PC 1', 'PC 2', 'PC 3'])
df_plot['Chunk Index'] = range(len(chunks)) # Add chunk index for identification
df_plot['Chunk Text'] = chunk_texts # Add the actual chunk text for hover data
df_plot['Source Document'] = chunk_sources # Add the source filename for coloring

# --- Add text wrapping to the Chunk Text for better hover display ---
wrapped_texts = []
wrapper = textwrap.TextWrapper(width=80, break_long_words=False, replace_whitespace=False) # Adjust width as needed
for text in df_plot['Chunk Text']:
    lines = wrapper.wrap(text)
    wrapped_texts.append("<br>".join(lines)) # Join lines with HTML break tag

df_plot['Wrapped Text'] = wrapped_texts
# --- End of text wrapping ---

# Create the 3D scatter plot, coloring by 'Source Document'
fig = px.scatter_3d(
    df_plot, x='PC 1', y='PC 2', z='PC 3',
    color='Source Document', # Color points by source document
    title=f'Embeddings of Document Chunks (Total Explained Variance: {total_var:.2f}%)',
    labels={'PC 1': 'Principal Component 1', 'PC 2': 'Principal Component 2', 'PC 3': 'Principal Component 3'},
    # Using hover_data to include data, then a custom hovertemplate to format it
    hover_data={'Chunk Index': True, 'Wrapped Text': False, 'Source Document': True} # Use Wrapped Text for hover, but format in template
)

# Create a custom hover template
# Use the 'Wrapped Text' column which already has <br> for line breaks
hover_template = '<b>Source:</b> %{customdata[2]}<br><b>Chunk Index:</b> %{customdata[0]}<br><br><b>Chunk Text:</b> %{customdata[1]}<extra></extra>'

# Update the traces to use the custom hovertemplate and include Wrapped Text and Source in customdata
fig.update_traces(
    hovertemplate=hover_template,
    customdata=np.stack((df_plot['Chunk Index'], df_plot['Wrapped Text'], df_plot['Source Document']), axis=-1)
)

# Adjust hover label layout (optional, might not affect custom hovertemplate much)
fig.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=10,
        # Setting a width here might still be useful in combination with text wrapping
        # width=300,
        # bordercolor="black"
    )
)


fig.show()

# 5. Configurando o Retrieval Chain

Agora que temos nossos documentos carregados, divididos em chunks, embaralhados (embeddings) e armazenados em um vetor store (`vectorstore`), podemos construir a **retrieval chain**.

Esta chain é o que conecta a pergunta do usuário com os documentos relevantes e, finalmente, com o modelo de linguagem para gerar a resposta.

Utilizaremos o `create_retrieval_chain` do LangChain, que combina um retriever (nosso `retriever`) com uma chain de documentos (`document_chain`) que já configuramos.

In [91]:
from langchain.chains.retrieval import create_retrieval_chain

# Crie a retrieval chain combinando o retriever e a document_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

A `retrieval_chain` agora pode receber uma pergunta e fará o seguinte internamente:

1.  Usa o `retriever` para buscar os documentos mais relevantes para a pergunta no `vectorstore`.
2.  Passa a pergunta original e os documentos recuperados para a `document_chain`.
3.  A `document_chain` usa o `llm_triagem` e o `prompt_rag` para gerar uma resposta baseada no contexto fornecido pelos documentos.

Vamos testar a chain!

In [93]:
# Ask a question using the retrieval chain
question = "Posso reembolsar a internet do meu home office?"
response = retrieval_chain.invoke({"input": question})

# Print the response
print("Question:", question)
print("Answer:", response["answer"])

# You can also inspect the retrieved documents
# print("\nRetrieved documents:")
# for doc in response["context"]:
#     print(doc)

Question: Posso reembolsar a internet do meu home office?
Answer: Sim, a internet para home office é reembolsável via subsídio, no valor de até R$ 100/mês, mediante nota fiscal nominal.


# Task
Integrate the `triagem` function and the `retrieval_chain` to create a system that first triages a user's message and then, if appropriate, uses the retrieval chain to answer the question.

## Review existing components

### Subtask:
Briefly review the `triagem` function and the `retrieval_chain` to understand their inputs and outputs.


**Reasoning**:
I need to examine the `triagem` function and the `retrieval_chain` to understand their inputs and outputs. I will print the function definitions and a sample output for each to understand their structure.



In [94]:
import inspect

# Examine the triagem function
print("--- triagem function signature ---")
print(inspect.signature(triagem))
print("\n--- Sample triagem output ---")
sample_triagem_output = triagem("Tenho uma dúvida sobre um benefício.")
print(sample_triagem_output)

print("\n--- retrieval_chain input expectation ---")
# The retrieval_chain is created using create_retrieval_chain, which expects a dictionary with 'input'
print("Expected input for retrieval_chain: {'input': 'user question'}")

print("\n--- Sample retrieval_chain output ---")
sample_retrieval_output = retrieval_chain.invoke({"input": "Posso reembolsar a internet do meu home office?"})
print(sample_retrieval_output)

--- triagem function signature ---
(mensagem: str) -> Dict

--- Sample triagem output ---
{'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['Qual benefício você se refere?']}

--- retrieval_chain input expectation ---
Expected input for retrieval_chain: {'input': 'user question'}

--- Sample retrieval_chain output ---
{'input': 'Posso reembolsar a internet do meu home office?', 'context': [Document(id='fc24183c-9ce9-46b1-932f-9d2a12a25e15', metadata={'producer': 'Skia/PDF m140 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '/tmp/tmpajz7rgts.pdf', 'file_path': '/tmp/tmpajz7rgts.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Imersão: Política de Reembolsos (Viagens e Despesas)', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='4.\u200b Internet para home office: reembolsável via subsídio'), Document(id='e1c6bff3-c9c0-45cd-9779-b1b500cc5d3c', metadata={'pro

## Create a combined function

### Subtask:
Develop a new function that takes a user message as input and integrates the triaging and RAG functionalities.


**Reasoning**:
Define the new function `handle_user_message` that calls `triagem` and returns its result as a first step towards integrating the functionalities.



In [95]:
def handle_user_message(user_message: str) -> Dict:
    """
    Handles a user message by first triaging it.

    Args:
        user_message: The input message from the user.

    Returns:
        A dictionary containing the triaging result.
    """
    triagem_result = triagem(user_message)
    return triagem_result

## Conditional rag execution

### Subtask:
Based on the `decisao` from the triaging step, conditionally execute the retrieval chain or return the triaging result.


**Reasoning**:
Modify the `handle_user_message` function to conditionally execute the retrieval chain based on the triaging result.



In [96]:
def handle_user_message(user_message: str) -> Dict:
    """
    Handles a user message by first triaging it and then, if appropriate,
    using the retrieval chain to answer the question.

    Args:
        user_message: The input message from the user.

    Returns:
        A dictionary containing the triaging result and potentially the RAG answer.
    """
    triagem_result = triagem(user_message)

    if triagem_result['decisao'] == 'AUTO_RESOLVER':
        # If the decision is AUTO_RESOLVER, invoke the retrieval chain
        rag_response = retrieval_chain.invoke({"input": user_message})
        # Return the triaging result and the RAG answer
        return {'triagem': triagem_result, 'rag_answer': rag_response['answer']}
    else:
        # If the decision is PEDIR_INFO or ABRIR_CHAMADO, just return the triaging result
        return {'triagem': triagem_result}


**Reasoning**:
Test the updated `handle_user_message` function with different inputs to ensure the conditional logic works correctly.



In [97]:
# Test with an AUTO_RESOLVER case
print("--- Testing AUTO_RESOLVER ---")
response_auto = handle_user_message("Posso reembolsar a internet do meu home office?")
print(response_auto)

print("\n--- Testing PEDIR_INFO ---")
# Test with a PEDIR_INFO case
response_info = handle_user_message("Tenho uma dúvida sobre um benefício.")
print(response_info)

print("\n--- Testing ABRIR_CHAMADO ---")
# Test with an ABRIR_CHAMADO case
response_chamado = handle_user_message("Quero mais 5 dias de trabalho remoto. Como faço?")
print(response_chamado)

--- Testing AUTO_RESOLVER ---
{'triagem': {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}, 'rag_answer': 'Sim, a internet para home office é reembolsável via subsídio, até R$ 100/mês, mediante nota fiscal nominal.'}

--- Testing PEDIR_INFO ---
{'triagem': {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['Qual benefício você se refere?']}}

--- Testing ABRIR_CHAMADO ---
{'triagem': {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}}


## Format the final output

### Subtask:
Structure the output of the combined function to clearly indicate the triaging decision and, if applicable, the answer from the RAG system.


**Reasoning**:
Update the `handle_user_message` function to ensure a consistent output structure for all triaging decisions and test the function with examples.



In [98]:
def handle_user_message(user_message: str) -> Dict:
    """
    Handles a user message by first triaging it and then, if appropriate,
    using the retrieval chain to answer the question.

    Args:
        user_message: The input message from the user.

    Returns:
        A dictionary containing the triaging result and potentially the RAG answer.
    """
    triagem_result = triagem(user_message)

    output = {'triagem': triagem_result} # Always include the triagem result

    if triagem_result['decisao'] == 'AUTO_RESOLVER':
        # If the decision is AUTO_RESOLVER, invoke the retrieval chain
        rag_response = retrieval_chain.invoke({"input": user_message})
        # Include the RAG answer under the 'rag_answer' key
        output['rag_answer'] = rag_response['answer']
    # For 'PEDIR_INFO' and 'ABRIR_CHAMADO', the output already contains only the triaging info

    return output

# Test the updated function with examples covering all three triaging decisions
print("--- Testing AUTO_RESOLVER ---")
response_auto = handle_user_message("Posso reembolsar a internet do meu home office?")
print(response_auto)

print("\n--- Testing PEDIR_INFO ---")
response_info = handle_user_message("Tenho uma dúvida sobre um benefício.")
print(response_info)

print("\n--- Testing ABRIR_CHAMADO ---")
response_chamado = handle_user_message("Quero mais 5 dias de trabalho remoto. Como faço?")
print(response_chamado)

--- Testing AUTO_RESOLVER ---
{'triagem': {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}, 'rag_answer': 'Sim, a internet para home office é reembolsável via subsídio, até R$ 100/mês, mediante nota fiscal nominal.'}

--- Testing PEDIR_INFO ---
{'triagem': {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['Qual benefício você se refere?']}}

--- Testing ABRIR_CHAMADO ---
{'triagem': {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}}


## Test the combined system

### Subtask:
Run test cases with different types of questions to ensure the combined logic works as expected.


**Reasoning**:
Define test questions and iterate through them, calling the handle_user_message function and printing the results to verify the combined logic.



In [99]:
test_questions = [
    "Posso reembolsar a internet do meu home office?", # AUTO_RESOLVER
    "Quero mais 5 dias de trabalho remoto. Como faço?", # ABRIR_CHAMADO
    "Tenho uma dúvida sobre um benefício.", # PEDIR_INFO
    "Posso reembolsar cursos ou treinamentos da Alura?", # AUTO_RESOLVER (should get RAG answer if in documents)
    "Quantas capivaras tem no Rio Pinheiros?" # Should be PEDIR_INFO as it's out of scope
]

for question in test_questions:
    print(f"--- Testing Question: '{question}' ---")
    result = handle_user_message(question)
    print(result)
    print("-" * 30)

    # Verification steps (basic checks)
    assert 'triagem' in result, "Output missing 'triagem' key"
    assert 'decisao' in result['triagem'], "Triagem result missing 'decisao' key"

    if result['triagem']['decisao'] == 'AUTO_RESOLVER':
        assert 'rag_answer' in result, "AUTO_RESOLVER missing 'rag_answer'"
        # You could add more specific checks here, e.g., if the answer contains expected keywords
    else:
        assert 'rag_answer' not in result, f"{result['triagem']['decisao']} should not have 'rag_answer'"
        if result['triagem']['decisao'] == 'PEDIR_INFO':
            assert 'campos_faltantes' in result['triagem'], "PEDIR_INFO missing 'campos_faltantes'"
        elif result['triagem']['decisao'] == 'ABRIR_CHAMADO':
             # Check for specific urgency or other expected fields if needed
             pass # No specific checks for ABRIR_CHAMADO needed for this subtask

print("\n✅ All test cases completed and basic assertions passed.")

--- Testing Question: 'Posso reembolsar a internet do meu home office?' ---
{'triagem': {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}, 'rag_answer': 'Sim, a internet para home office é reembolsável via subsídio, até R$ 100/mês, mediante nota fiscal nominal.'}
------------------------------
--- Testing Question: 'Quero mais 5 dias de trabalho remoto. Como faço?' ---
{'triagem': {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []}}
------------------------------
--- Testing Question: 'Tenho uma dúvida sobre um benefício.' ---
{'triagem': {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': ['qual benefício', 'qual a dúvida']}}
------------------------------
--- Testing Question: 'Posso reembolsar cursos ou treinamentos da Alura?' ---
{'triagem': {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}, 'rag_answer': 'Não sei.'}
------------------------------
--- Testing Question: 'Quantas capivaras tem no Rio

## Summary:

### Data Analysis Key Findings

*   The `triagem` function takes a user message as input and returns a dictionary containing a `decisao` (decision) which can be 'AUTO\_RESOLVER', 'PEDIR\_INFO', or 'ABRIR\_CHAMADO', along with other potential keys like `urgencia` and `campos_faltantes`.
*   The `retrieval_chain` is designed to answer questions based on provided documents and expects a dictionary input with the key 'input' containing the user's query. It outputs a dictionary including the `answer`.
*   A combined function `handle_user_message` was created to first call `triagem` with the user's message.
*   Based on the `decisao` from the triaging step:
    *   If the decision is 'AUTO\_RESOLVER', the `retrieval_chain` is invoked with the user's message, and the output includes both the triaging result and the RAG answer under the key 'rag\_answer'.
    *   If the decision is 'PEDIR\_INFO' or 'ABRIR\_CHAMADO', the `retrieval_chain` is not invoked, and the output only contains the triaging result.
*   The final output structure of `handle_user_message` is a dictionary that always contains the triaging result under the key 'triagem' and conditionally includes the RAG answer under the key 'rag\_answer' for 'AUTO\_RESOLVER' cases.
*   Testing with various questions confirmed that the system correctly routes messages based on the triaging decision and provides a RAG answer only when appropriate.

### Insights or Next Steps

*   The current system effectively routes user queries based on a triaging step, preventing unnecessary RAG calls for questions requiring more information or ticket creation.
*   Further refinement of the `triagem` function's logic and the RAG system's document set could improve the accuracy of the triaging decisions and the quality of the answers for 'AUTO\_RESOLVER' cases.
